In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from datetime import datetime

In [2]:
# os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

load voter data

In [3]:
# voter = pd.read_csv('20220512_voter_race_location.csv', low_memory = False)
voter = pd.read_csv('20220516_voter_race_location.csv', low_memory = False)

In [4]:
voter1 = voter[pd.isna(voter.lat) == False].copy()

In [5]:
len(voter1)/len(voter)

0.9977798137731331

In [6]:
addresses = voter1.iloc[:,[4,5,6,7,8,9]].copy().drop_duplicates()
addressesgdf = gpd.GeoDataFrame(addresses, geometry=gpd.points_from_xy(addresses['long'], addresses['lat']))
addressesgdf.crs = 'EPSG:4326'

In [7]:
del voter
del voter1

load isochrones

In [8]:
# drivingchrones = gpd.read_file('20220623_driving_isochrones.shp')
drivingchrones = gpd.read_file('20220627-driving1.shp')
transitchrones = gpd.read_file('20220623_transit_isochrones_2.shp')
drivingchrones.lat = np.round(drivingchrones.lat,6)
drivingchrones.long = np.round(drivingchrones.long,6)
transitchrones.lat = transitchrones.lat.astype('float')
transitchrones.long = transitchrones.long.astype('float')

In [9]:
inyear = transitchrones[['in_2022','in_2021','in_nov_202','lat','long']].copy().drop_duplicates()

In [10]:
drivingchrones.head()

,travel_tim,lat,long,GEOID,in_2022,in_nov_202,geometry
0,300,34.915487,-85.106602,13047,1,1,"POLYGON ((-85.10147 34.92004, -85.10284 34.920..."
1,600,34.915487,-85.106602,13047,1,1,"MULTIPOLYGON (((-85.10789 34.98658, -85.10713 ..."
2,900,34.915487,-85.106602,13047,1,1,"MULTIPOLYGON (((-85.22371 34.98556, -85.22355 ..."
3,1200,34.915487,-85.106602,13047,1,1,"POLYGON ((-84.97980 34.98292, -84.97978 34.981..."
4,1500,34.915487,-85.106602,13047,1,1,"POLYGON ((-85.14537 34.76766, -85.14644 34.767..."


In [11]:
drivingvoterjoin = gpd.sjoin(addressesgdf,drivingchrones, how = 'left')

In [12]:
drivingvoterjoin1 = drivingvoterjoin.drop('geometry', axis = 1)

In [13]:
drivingvoterjoin2 =drivingvoterjoin1.drop_duplicates()

In [14]:
drivingvoterjoin2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79168676 entries, 0 to 7545312
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   lat_left         float64
 5   long_left        float64
 6   index_right      float64
 7   travel_tim       float64
 8   lat_right        float64
 9   long_right       float64
 10  GEOID            object 
 11  in_2022          object 
 12  in_nov_202       object 
dtypes: float64(6), object(7)
memory usage: 8.3+ GB


In [15]:
drivingvoterjoin2.to_csv('20220627_voter_address_driving_isochrone_join.csv')

In [16]:
del drivingchrones
# del drivingchrones1

In [17]:
transitjoin = gpd.sjoin(addressesgdf, transitchrones, how = 'left')

/Users/knakajima/opt/anaconda3/envs/geo_env_dropbox_2/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  """Entry point for launching an IPython kernel.


In [18]:
transitjoin1 = transitjoin.drop('geometry', axis = 1)

In [19]:
transitjoin1.to_csv('20220627_voter_addr_transitchrone_join.csv')

joining addresses with geographies

In [20]:
urban = gpd.read_file('tl_2021_us_uac10/tl_2021_us_uac10.shp')
GA = urban[urban.NAME10.str.contains('GA')].copy()

In [21]:
GAurban = GA[['geometry']].copy().reset_index()

In [22]:
GAurban['urban'] = True

In [23]:
GAurban=GAurban.to_crs('EPSG:4326')

In [24]:
GAurban1 = GAurban.drop('index', axis = 1)

In [25]:
GAurban1.head()

,geometry,urban
0,"POLYGON ((-82.71957 33.72904, -82.71805 33.728...",True
1,"MULTIPOLYGON (((-82.60097 31.84351, -82.60098 ...",True
2,"POLYGON ((-81.91131 32.15790, -81.91141 32.156...",True
3,"MULTIPOLYGON (((-83.44627 31.11137, -83.44635 ...",True
4,"MULTIPOLYGON (((-82.33667 32.20602, -82.33522 ...",True


In [26]:
# referencing urban area
addrjoin1 = gpd.sjoin(addressesgdf, GAurban1, how = 'left')

In [27]:
addrjoin1.urban.value_counts()

True    2268938
Name: urban, dtype: int64

In [28]:
2269538/len(addrjoin1)

0.7112579323753737

In [29]:
tracts = gpd.read_file('tl_2021_13_tract/tl_2021_13_tract.shp')

In [30]:
tracts = tracts.to_crs('EPSG:4326')

In [31]:
addrjoin1 = addrjoin1.drop('index_right', axis = 1)

In [32]:
addrjoin2 = gpd.sjoin(addrjoin1, tracts, how = 'left')

In [33]:
addrjoin2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3190903 entries, 0 to 7545312
Data columns (total 21 columns):
 #   Column           Dtype   
---  ------           -----   
 0   st_address       object  
 1   RESIDENCE_CITY   object  
 2   RESIDENCE_STATE  object  
 3   ZIP5             object  
 4   lat              float64 
 5   long             float64 
 6   geometry         geometry
 7   urban            object  
 8   index_right      float64 
 9   STATEFP          object  
 10  COUNTYFP         object  
 11  TRACTCE          object  
 12  GEOID            object  
 13  NAME             object  
 14  NAMELSAD         object  
 15  MTFCC            object  
 16  FUNCSTAT         object  
 17  ALAND            float64 
 18  AWATER           float64 
 19  INTPTLAT         object  
 20  INTPTLON         object  
dtypes: float64(5), geometry(1), object(15)
memory usage: 535.6+ MB


In [35]:
addrjoin2.drop('geometry', axis = 1).to_csv('20220627_voter_address_tract_ua.csv')